In [2]:
# Importing the utilites
import nltk

from nltk.stem import WordNetLemmatizer

import spacy
from scipy.sparse import vstack
import json
import pickle
import yaml

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import tensorflow as tf 
import re

In [3]:
words = []
classes = []
documents = []
ignore_words = ['!','?','#','.']

In [4]:
# loading the intents.yml file
with open('intents.yml', 'r',encoding='utf-8') as file:
    yaml_data = yaml.safe_load(file)
intents = yaml_data['intents']
intents    

[{'tag': 'greeting',
  'patterns': ['Greetings',
   'Hey, chatbot',
   'Hi',
   'Hi there',
   'How are you',
   'Is anyone there?',
   'Hey',
   'Hola',
   'Hello',
   'Good day'],
  'responses': ['Hello, thanks for asking',
   'Good to see you again',
   'Hi there, how can I help?',
   "Hey, what's up?",
   "Hello! I'm here to help you. What can I do for you today?"],
  'context': []},
 {'tag': 'goodbye',
  'patterns': ['Bye',
   'See you later',
   'Goodbye',
   'Nice chatting to you, bye',
   'Till next time',
   'Catch you later',
   'See you soon',
   'Goodnight'],
  'responses': ['See you!',
   'Goodbye! It was a pleasure assisting you',
   'Have a nice day',
   'Bye! Come back again soon.',
   'Goodbye! Have a great day!',
   'Goodbye! Feel free to return if you have more questions',
   "Farewell! If you need anything else, don't hesitate to ask",
   "Take care! If you have more questions, I'll be here to help."],
  'context': []},
 {'tag': 'thanks',
  'patterns': ['Thanks',
  

<h4>Tokenization</h4>

- use spacy for Tokenzation.

In [5]:
# Load the English language model
# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])

In [6]:
for intent in intents:
    for pattern in intent['patterns']:
        # Tokenize each word and perform lemmatization
        tokenized_words =  nlp(pattern)
        words.extend(tokenized_words)
        # Add documents to the corpus
        documents.append((tokenized_words, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
print(words)
print( 'length',len(words))


[Greetings, Hey, ,, chatbot, Hi, Hi, there, How, are, you, Is, anyone, there, ?, Hey, Hola, Hello, Good, day, Bye, See, you, later, Goodbye, Nice, chatting, to, you, ,, bye, Till, next, time, Catch, you, later, See, you, soon, Goodnight, Thanks, Thank, you, That, 's, helpful, Awesome, ,, thanks, Thanks, for, helping, me, Thank, you, very, much, I, appreciate, it, Thanks, for, your, time, /, help, You, 're, great, ,, thanks, !, How, you, could, help, me, ?, What, you, can, do, ?, What, help, you, provide, ?, How, you, can, be, helpful, ?, What, support, is, offered, Where, to, apply, for, internship, ?, Internships, offered, How, to, apply, for, an, internship, ?, How, can, I, apply, for, the, internship, ?, How, can, I, get, more, information, about, the, internships, ?, How, long, the, internship, can, be, ?, What, is, the, duration, of, the, internship, ?, How, many, fields, are, there, ?, Which, field, that, the, internship, concerns, about, ?, Tell, me, about, internship, domains, 

<h4>Lemmatization</h4>

In [7]:
lemmatized_words = [token.lemma_ for token in nlp(" ".join(map(str, words)))]
# Update the 'words' list with the lemmatized forms
words = lemmatized_words
print(len(words))
words






262


['greeting',
 'hey',
 ',',
 'chatbot',
 'hi',
 'hi',
 'there',
 'how',
 'be',
 'you',
 'be',
 'anyone',
 'there',
 '?',
 'hey',
 'Hola',
 'hello',
 'good',
 'day',
 'bye',
 'see',
 'you',
 'later',
 'Goodbye',
 'Nice',
 'chatting',
 'to',
 'you',
 ',',
 'bye',
 'till',
 'next',
 'time',
 'catch',
 'you',
 'later',
 'see',
 'you',
 'soon',
 'Goodnight',
 'thank',
 'thank',
 'you',
 'that',
 'be',
 'helpful',
 'Awesome',
 ',',
 'thank',
 'thank',
 'for',
 'help',
 'I',
 'thank',
 'you',
 'very',
 'much',
 'I',
 'appreciate',
 'it',
 'thank',
 'for',
 'your',
 'time',
 '/',
 'help',
 'you',
 'be',
 'great',
 ',',
 'thank',
 '!',
 'how',
 'you',
 'could',
 'help',
 'I',
 '?',
 'what',
 'you',
 'can',
 'do',
 '?',
 'what',
 'help',
 'you',
 'provide',
 '?',
 'how',
 'you',
 'can',
 'be',
 'helpful',
 '?',
 'what',
 'support',
 'be',
 'offer',
 'where',
 'to',
 'apply',
 'for',
 'internship',
 '?',
 'internship',
 'offer',
 'how',
 'to',
 'apply',
 'for',
 'an',
 'internship',
 '?',
 'how',


In [8]:
lemmatized_sentence = " ".join(lemmatized_words)
doc = nlp(lemmatized_sentence)

# Extract named entities from the lemmatized sentence
named_entities = [(ent.text, ent.label_) for ent in doc.ents]
named_entities

[]

In [9]:

classes = sorted(list(set(classes)))
classes

['About CodeClause',
 'Contact',
 'goodbye',
 'greeting',
 'internships',
 'internships duration',
 'internships fields',
 'offer_letter',
 'options',
 'thanks']

In [10]:
print (len(classes), "classes", classes)

10 classes ['About CodeClause', 'Contact', 'goodbye', 'greeting', 'internships', 'internships duration', 'internships fields', 'offer_letter', 'options', 'thanks']


In [11]:
print (len(words), "unique lemmatized words", words)

262 unique lemmatized words ['greeting', 'hey', ',', 'chatbot', 'hi', 'hi', 'there', 'how', 'be', 'you', 'be', 'anyone', 'there', '?', 'hey', 'Hola', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'till', 'next', 'time', 'catch', 'you', 'later', 'see', 'you', 'soon', 'Goodnight', 'thank', 'thank', 'you', 'that', 'be', 'helpful', 'Awesome', ',', 'thank', 'thank', 'for', 'help', 'I', 'thank', 'you', 'very', 'much', 'I', 'appreciate', 'it', 'thank', 'for', 'your', 'time', '/', 'help', 'you', 'be', 'great', ',', 'thank', '!', 'how', 'you', 'could', 'help', 'I', '?', 'what', 'you', 'can', 'do', '?', 'what', 'help', 'you', 'provide', '?', 'how', 'you', 'can', 'be', 'helpful', '?', 'what', 'support', 'be', 'offer', 'where', 'to', 'apply', 'for', 'internship', '?', 'internship', 'offer', 'how', 'to', 'apply', 'for', 'an', 'internship', '?', 'how', 'can', 'I', 'apply', 'for', 'the', 'internship', '?', 'how', 'can', 'I', 'get', 'more

In [12]:
unique_words = []
for x in words:
        # check if exists in unique_list or not
        if x not in unique_words:
            unique_words.append(x)
            
print (len(unique_words), "unique lemmatized words", unique_words)         

82 unique lemmatized words ['greeting', 'hey', ',', 'chatbot', 'hi', 'there', 'how', 'be', 'you', 'anyone', '?', 'Hola', 'hello', 'good', 'day', 'bye', 'see', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'till', 'next', 'time', 'catch', 'soon', 'Goodnight', 'thank', 'that', 'helpful', 'Awesome', 'for', 'help', 'I', 'very', 'much', 'appreciate', 'it', 'your', '/', 'great', '!', 'could', 'what', 'can', 'do', 'provide', 'support', 'offer', 'where', 'apply', 'internship', 'an', 'the', 'get', 'more', 'information', 'about', 'long', 'duration', 'of', 'many', 'field', 'which', 'concern', 'tell', 'domain', 'codeclause', 'service', 'CodeClause', 'have', 'when', 'will', 'receive', 'my', 'letter', 'not', 'contact', 'in', 'touch', 'with']


In [13]:
# all possible sentences belonging to each class
print (len(documents), "documents")

62 documents


Creating Training data with  Intent classification



In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [15]:
# Prepare the data for TF-IDF
sentences = [" ".join([token.text for token in pattern]) for pattern, tag in documents]
labels = [tag for pattern, tag in documents]

# sentences ----> contain all the possible ways of sentences input by user
# labels --> labels will contain the corresponding intent tags.

In [ ]:
labels

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer()

# Example text documents
documents = sentences

# Fit and transform the documents
tfidf_matrix = vectorizer.fit_transform(documents)

# Get the feature names (words)
feature_names = vectorizer.get_feature_names()

# Print the feature names and TF-IDF matrix
print("Feature Names:", feature_names)
print("TF-IDF Matrix:")
print(tfidf_matrix.toarray())

Feature Names: ['about', 'an', 'anyone', 'applied', 'apply', 'appreciate', 'are', 'awesome', 'be', 'bye', 'can', 'catch', 'chatbot', 'chatting', 'codeclause', 'concerns', 'contact', 'could', 'day', 'do', 'domains', 'duration', 'field', 'fields', 'for', 'get', 'good', 'goodbye', 'goodnight', 'great', 'greetings', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'in', 'information', 'internship', 'internships', 'is', 'it', 'later', 'letter', 'long', 'many', 'me', 'more', 'much', 'my', 'next', 'nice', 'not', 'of', 'offer', 'offered', 'provide', 're', 'receive', 'received', 'see', 'services', 'soon', 'support', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'touch', 'very', 'what', 'when', 'where', 'which', 'will', 'with', 'you', 'your']
TF-IDF Matrix:
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.  

c:\Users\Lakshya\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [18]:
tf_matrix = tfidf_matrix.toarray()
tf_matrix
# Initialize the vector object
# vectorizer = TfidfVectorizer(lowercase=True)

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.43745151, 0.28354449,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.43533937, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.43533937, 0.        ,
        0.        ]])

In [19]:
# fit vectorizer on the sentences
# tfidf_features = vectorizer.fit_transform(sentences)
# tfidf_features

In [20]:
# feature_names = vectorizer.get_feature_names()
# print(feature_names)


In [58]:
training=[]
output_empty = [0]*len(classes)
len(classes)

10

In [67]:
# create  a training set 
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [token.lemma_ for token in nlp(" ".join(map(str, words)))]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
     


ValueError: 'r' is not in list

In [69]:
print(pattern_words)

['greet', 'hey', ',', 'chatbot', 'hi', 'hi', 'there', 'how', 'be', 'you', 'be', 'anyone', 'there', '?', 'hey', 'Hola', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'till', 'next', 'time', 'catch', 'you', 'later', 'see', 'you', 'soon', 'Goodnight', 'thank', 'thank', 'you', 'that', 'be', 'helpful', 'Awesome', ',', 'thank', 'thank', 'for', 'help', 'I', 'thank', 'you', 'very', 'much', 'I', 'appreciate', 'it', 'thank', 'for', 'your', 'time', '/', 'help', 'you', 'be', 'great', ',', 'thank', '!', 'how', 'you', 'could', 'help', 'I', '?', 'what', 'you', 'can', 'do', '?', 'what', 'help', 'you', 'provide', '?', 'how', 'you', 'can', 'be', 'helpful', '?', 'what', 'support', 'be', 'offer', 'where', 'to', 'apply', 'for', 'internship', '?', 'internship', 'offer', 'how', 'to', 'apply', 'for', 'an', 'internship', '?', 'how', 'can', 'I', 'apply', 'for', 'the', 'internship', '?', 'how', 'can', 'I', 'get', 'more', 'information', 'about', 'the

In [23]:
# Convert Training Set to Numpy Arrays:
training_set = np.array(training_set)
X_train = np.array(training_set[:, 0].tolist())
y_train = np.array(training_set[:, 1].tolist())

C:\Users\Lakshya\AppData\Local\Temp\ipykernel_1944\3934461316.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_set = np.array(training_set)


In [24]:
X_train.shape

(3844, 87)

In [25]:
print("TF-IDF Features (X_train):\n", X_train)
print("Intent Labels (y_train):\n", y_train)

TF-IDF Features (X_train):
 [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.43745151 0.28354449 0.        ]
 [0.         0.         0.         ... 0.43533937 0.         0.        ]
 [0.         0.         0.         ... 0.43533937 0.         0.        ]]
Intent Labels (y_train):
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


<h4>Model</h4>

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import LearningRateScheduler

In [27]:
# create neural network model using the Keras framework with TensorFlow backend
# This model is used for intent classification based on the TF-IDF features (train_x) and one-hot encoded intent labels (train_y).
# model = Sequential()
# model.add(Dense(128, input_shape=(X_train.shape[0],), activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(len(y_train[0]), activation='softmax'))


In [28]:
X_train.shape

(3844, 87)

In [29]:
# stacked_sparse_matrix = vstack(X_train)

# Convert the stacked sparse matrix to a dense NumPy array
# dense_array = stacked_sparse_matrix.toarray()

# Create the Keras Sequential model
if len(_train.shape) > 2:
    X_train = X_train_tensor.reshape(x_train_tensor.shape[0], -1)
else:
    print("all ok")    
    
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

# Compile the model with the optimizer you prefer
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Convert the dense array to a tensor using tf.convert_to_tensor
x_train_tensor = tf.convert_to_tensor(dense_array, dtype=tf.float32)

# Fit the model with the tensor input
history = model.fit(x_train_tensor, y_train, epochs=200, batch_size=5, verbose=1)

# Save the model
model.save('bot_model.h5')

NameError: name '_train' is not defined

Conclusion:
- model is successfully trained now

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import json
import yaml
import random
from keras.models import load_model
     

In [ ]:

model = load_model('bot_model.h5')
# intents = json.loads(open('intents.yml').read())
with open('intents.yml', 'r',encoding='utf-8') as file:
    yaml_data = yaml.safe_load(file)
intents = yaml_data['intents']  
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
    

In [ ]:
doc = "dragon fox in the house of batiatus"
sentence_word = nltk.word_tokenize(doc)
sentence_word

['dragon', 'fox', 'in', 'the', 'house', 'of', 'batiatus']

In [ ]:
def pre_process(sentence):
    # Tokenize the pattern - split into segments
    nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])
    sentenceWords = nlp(sentence)
    # tem each word - create short form for word
    sentenceWords = [token.lemma_ for token in nlp(" ".join(map(str, sentenceWords)))]
    # Update the 'words' list with the lemmatized forms
    return sentenceWords

In [ ]:
# TF-IDF vectorizer: 0 or 1 for each word in the vector that exists in the sentence
def tfvectorise(sentence, vectorizer, show_details=True):
    # tokenize the pattern
    sentence_words = pre_process(sentence)
    # fitted
    vectorizer.fit(sentence_words)
    # TfidfVectorizer to transform the sentence into a TF-IDF vector
    tfidf_vector = vectorizer.transform([sentence]).toarray()
    if show_details:
        print("TF-IDF Vector:")
        print(tfidf_vector)
    return (np.array(tfidf_vector))



# def tfvectorise(sentence, vectorizer, show_details=True):
#     # tokenize the pattern
#     sentence_words = pre_process(sentence)
#     # fitted
#     vectorizer.fit(sentence_words)
#     # TfidfVectorizer to transform the sentence into a TF-IDF vector
#     tfidf_vector = vectorizer.transform([sentence]).toarray().flatten()  # Reshape to (87,) array
#     if show_details:
#         print("TF-IDF Vector:")
#         print(tfidf_vector)
#     return tfidf_vector

In [ ]:
# vectorizer = TfidfVectorizer()
# doc = "dragon in the house dragon  please help"
# tfvectorise(doc,vectorizer,True)

In [ ]:
# This function is used to predict the intent of a given sentence based on the trained model and TF-IDF vector of the sentence.
# # The intent with the highest probability above the error threshold is considered the final prediction.
# def predict_class(sentence, model):
#     x = tfvectorise(sentence,vectorizer,show_details=False)
#     res = model.predict(np.array([x]))[0]
#     ERROR_THRESHOLD = 0.25
#     results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
#     # sort by strength of probability
#     results.sort(key=lambda x: x[1], reverse=True)
#     return_list = []
#     for r in results:
#         return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
#     return return_list
    
    
def predict_class(sentence, model):
    x = tfvectorise(sentence, vectorizer, show_details=False)
    res = model.predict(np.array([x]))[0]  # Reshape to (1, 87)
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [ ]:
# retrieve a random response associated with the predicted intent.
def getResponse(ints, intents_yml):
    tag = ints[0]['intent']
    list_of_intents = intents_yml['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
# handling user input, processing it, and displaying the chatbot's response in the chat log window
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

In [ ]:
def start_chat():
     name = input("Enter Your Name: ")
     print("Welcome " + name + " to the Code Clause Bot Service! Let me know how can I help you?\n")
     while True:
         input_msg = str(input()).lower()
         if input_msg.lower()=="end":
             break
         if input_msg.lower()== '' or input_msg.lower()== '*':
             print('Please re-phrase your query!')
             print("-"*50)
         else:
             print(f"Code Clause: {chatbot_response(input_msg)}"+'\n')
             print("-"*50)

# start the chat bot
start_chat()

In [ ]:
def start_chat():
     name = input("Enter Your Name: ")
     print("Welcome " + name + " to the Code Clause Bot Service! Let me know how can I help you?\n")
     while True:
         input_msg = input("You: ").lower()
         if input_msg == "end":
             print("Chat ended.")
             break
         if input_msg == '' or input_msg == '*':
             print('Please re-phrase your query!')
         else:
             response = chatbot_response(input_msg)
             print(f"Code Clause: {response}\n")
         print("-" * 50)

In [ ]:
start_chat()

Welcome hi to the Code Clause Bot Service! Let me know how can I help you?



ValueError: in user code:

    File "C:\Users\Lakshya\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Lakshya\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Lakshya\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Lakshya\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "C:\Users\Lakshya\AppData\Roaming\Python\Python39\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Lakshya\AppData\Roaming\Python\Python39\site-pack

In [ ]:
import tkinter
from tkinter import *
import sys
import os
if os.environ.get('DISPLAY','')=='':
  print('no DISPLAY found. Using: 0.0')
  os.environ.__setitem__('DISPLAY',':0.0')
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()